### Main code

In [2]:
from typing import List
import numpy as np


class InputStream:
    def __init__(self, data):
        self.data = data
        self.i = 0

    def read(self, size):
        out = self.data[self.i:self.i + size]
        self.i += size
        return int(out, 2)


def access_bit(data, num):
    """ from bytes array to bits by num position"""
    base = int(num // 8)
    shift = 7 - int(num % 8)
    return (data[base] & (1 << shift)) >> shift


def bytes2bit(data):
    """ get bit string from bytes data"""
    return ''.join([str(access_bit(data, i)) for i in range(len(data) * 8)])


def rle_to_mask(rle: List[int], height: int, width: int) -> np.array:
    """
    Converts rle to image mask
    Args:
        rle: your long rle
        height: original_height
        width: original_width

    Returns: np.array
    """

    rle_input = InputStream(bytes2bit(rle))

    num = rle_input.read(32)
    word_size = rle_input.read(5) + 1
    rle_sizes = [rle_input.read(4) + 1 for _ in range(4)]
    # print('RLE params:', num, 'values,', word_size, 'word_size,', rle_sizes, 'rle_sizes')

    i = 0
    out = np.zeros(num, dtype=np.uint8)
    while i < num:
        x = rle_input.read(1)
        j = i + 1 + rle_input.read(rle_sizes[rle_input.read(2)])
        if x:
            val = rle_input.read(word_size)
            out[i:j] = val
            i = j
        else:
            while i < j:
                val = rle_input.read(word_size)
                out[i] = val
                i += 1

    image = np.reshape(out, [height, width, 4])[:, :, 3]
    return image

In [5]:
from PIL import Image

class LSbrush_2_SematicSegm:
    def __init__(self, full_path):
        self.classes = {"leaf" :1, "tomato":2, "stem":3}
        self.full_path = full_path
        
    def run(self, item_label):
        original_width = item_label["annotations"][0]["result"][0]["original_width"]
        original_height = item_label["annotations"][0]["result"][0]["original_height"]

        # Base name
        image_name = item_label['data']['image']
        image_name = os.path.basename(image_name)

        np_array_base = np.zeros([original_height, original_width], dtype=np.uint8)
        
        for result in item_label["annotations"][0]["result"]:
            rle = result["value"]["rle"]
            label = result["value"]["brushlabels"][0]
            value_label = self.classes[label]

            image_np = rle_to_mask( rle,                               # result['value']['rle']
                                    original_height,                   # result['original_height']
                                    original_width).astype(np.uint8)   # result['original_width']  
            
            np_array_base[image_np != 0] = value_label
            np_array_base=np_array_base.astype(np.uint8)
        
        full_path = os.path.join(self.full_path, image_name)
        Image.fromarray(np_array_base).save(full_path)
                 

In [6]:
import json
import os

path_labeled = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/010_data_greenhouse_20_12_23/selected_images/Olga/project-21-at-2024-01-29-09-05-9fc962dd.json"
full_path = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/010_data_greenhouse_20_12_23/selected_images/Olga/mask_labeles"

with open(path_labeled) as user_file:
    parsed_json = json.load(user_file)

for item_label in parsed_json:
    c=LSbrush_2_SematicSegm(full_path)
    c.run(item_label)
